In [22]:
using Revise
using BilevelTrajOpt

using RigidBodyDynamics
using Plots
using ForwardDiff

In [10]:
urdf = joinpath("..", "urdf", "ball.urdf")
mechanism = parse_urdf(Float64, urdf)
body = findbody(mechanism, "ball")
basejoint = joint_to_parent(body, mechanism)
floatingjoint = Joint(basejoint.name, frame_before(basejoint), frame_after(basejoint), QuaternionFloating{Float64}())
replace_joint!(mechanism, basejoint, floatingjoint)
position_bounds(floatingjoint) .= RigidBodyDynamics.Bounds(-100, 100)
velocity_bounds(floatingjoint) .= RigidBodyDynamics.Bounds(-100, 100)
μ = 0.5
motion_type = :xyz
point = Point3D(default_frame(findbody(mechanism, "floor")), [0.,0.,0.])
normal = FreeVector3D(default_frame(findbody(mechanism, "floor")), [0.,0.,1.])
hs = HalfSpace(point, normal)
floor = Obstacle(hs, μ, motion_type)
obstacles = [floor]
env = parse_contacts(mechanism, urdf, obstacles)
x0 = MechanismState(mechanism)
Δt = 0.005
sim_data = get_sim_data(x0,env,Δt,true);

In [37]:
q0 = [1., 0., 0., 0., 0., 0., 0.]
v0 = [0., 0., 0., 0.1, 0.1, 0.]

u0 = zeros(sim_data.num_v)
set_configuration!(x0, q0)
set_velocity!(x0, v0)
setdirty!(x0)
ctrl! = (u,t,x) -> u[:] .= 0.
traj = BilevelTrajOpt.simulate(x0,env,sim_data.Δt,1,ctrl!,implicit_contact=false)
qnext = traj[1:sim_data.num_q,2]
vnext = traj[sim_data.num_q+1:sim_data.num_q+sim_data.num_v,2]
x_sol_exp = traj[sim_data.num_q+sim_data.num_v+2:end,2]

# qnext = q0
# vnext = v0

τ_ip, x_sol_ip, λ_ip, μ_ip, fx_ip = solve_implicit_contact_τ(sim_data,q0,v0,u0,qnext,vnext,ip_method=true);

τ_auglag, x_sol_auglag, λ_auglag, μ_auglag, fx_auglag = solve_implicit_contact_τ(sim_data,q0,v0,u0,qnext,vnext,ip_method=false);

display("Explicit")
display(x_sol_exp)
display("IP")
display(x_sol_ip)
display("Aug Lag")
display(x_sol_auglag)

display("torques")
display(τ_ip)
display(τ_auglag)

elapsed time: 0.005247976 seconds
elapsed time: 0.004505803 seconds
elapsed time: 0.0043502 seconds
elapsed time: 0.004263252 seconds
elapsed time: 0.004611853 seconds
elapsed time: 0.004364134 seconds
elapsed time: 0.004770623 seconds
elapsed time: 0.006635372 seconds
elapsed time: 0.004639891 seconds
elapsed time: 0.005896255 seconds
elapsed time: 0.004624777 seconds
elapsed time: 0.004699098 seconds


"Explicit"

6-element Array{Float64,1}:
 5.588405323632131e-8 
 5.5884053236320544e-8
 0.24998969429556853  
 0.24998969429557005  
 0.08773801440638201  
 0.9999592255910942   

"IP"

6-element Array{Float64,1}:
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0

...

6-element Array{Float64,1}:
 1.8026632154023182e-8
 1.8315795910286483e-8
 0.24998977312762763  
 0.2499897739252708   
 0.08773800822925623  
 0.9999591671684256   

"torques"

6-element Array{Float64,1}:
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0

6-element Array{Float64,1}:
  0.0               
  0.0               
  0.0               
  2.452399497540766 
  2.4523995025289484
 -9.809599429922255 

elapsed time: 0.005774063 seconds
elapsed time: 0.008975317 seconds
elapsed time: 0.004631043 seconds
elapsed time: 0.004988082 seconds
elapsed time: 0.004403167 seconds
elapsed time: 0.00424253 seconds
elapsed time: 0.004186848 seconds
elapsed time: 0.004249571 seconds
elapsed time: 0.004496583 seconds
Solve_Succeeded
Not_Enough_Degrees_Of_Freedom


In [38]:
τ_auglag, x_sol_auglag, λ_auglag, μ_auglag, fx_auglag = solve_implicit_contact_τ(sim_data,q0,v0,u0,qnext,vnext,ip_method=false);

# autodiff 
J_auto = ForwardDiff.jacobian(qv -> solve_implicit_contact_τ(sim_data,q0,v0,u0,qv[1:sim_data.num_q],qv[sim_data.num_q+1:sim_data.num_q+sim_data.num_v],ip_method=false)[2], vcat(qnext,vnext))
# Jt_auto = ForwardDiff.jacobian(qv -> solve_implicit_contact_τ(sim_data,q0,v0,u0,qv[1:sim_data.num_q],qv[sim_data.num_q+1:sim_data.num_q+sim_data.num_v],ip_method=false)[1], vcat(qnext,vnext))

# numerical
ϵ = sqrt(eps(1.))
J_num = zeros(size(J_auto))
# Jt_num = zeros(size(J_auto))
# Jt_num_ip = zeros(size(J_auto))
qv = vcat(qnext,vnext)
for i = 1:length(qv)
    δ = zeros(length(qv))
    δ[i] = ϵ 
    τ, x, λ, μ, fx = solve_implicit_contact_τ(sim_data,q0,v0,u0,(qv+δ)[1:sim_data.num_q],(qv+δ)[sim_data.num_q+1:sim_data.num_q+sim_data.num_v],ip_method=false)
    J_num[:,i] = (x - x_sol_auglag) ./ ϵ
#     Jt_num[:,i] = (τ - τ_auglag) ./ ϵ
#     τ, x, λ, μ, fx = solve_implicit_contact_τ(sim_data,q0,v0,u0,(qv+δ)[1:sim_data.num_q],(qv+δ)[sim_data.num_q+1:sim_data.num_q+sim_data.num_v],ip_method=true)
#     Jt_num_ip[:,i] = (τ - τ_auglag) ./ ϵ
end

display(J_auto)
display(J_num)

# display(Jt_auto)
# display(Jt_num)
# display(Jt_num_ip)

6×13 Array{Float64,2}:
  0.0543111  -0.0135724    0.0135723   …    1.10732    1.10718   0.553611  
  0.0543052  -0.0135717    0.0135718        1.10714    1.10716   0.553587  
 -0.0981399  -0.100421     0.100502       -12.1945     8.192     4.09613   
 -0.0981458  -0.100501     0.100421         8.19185  -12.1945    4.0961    
  0.175476    1.99721e-6  -1.99687e-6       0.5        0.5       1.22175e-7
 -0.17368    -0.456547     0.456548    …   -3.54249   -3.54254  18.6158    

6×13 Array{Float64,2}:
  0.0447884   0.0131502    …    1.12373     1.13101    0.541056  
  0.0758898   0.00187172        1.12597     1.14592    0.560105  
 -0.0999753  -0.0816865       -12.2363      8.14572    4.06829   
 -0.123324   -0.197803          8.21562   -12.2106     4.10294   
  0.173808   -0.000345013       0.500185    0.500993   0.00253675
 -0.202783   -0.519097     …   -3.50186    -3.5375    18.5688    